In [1]:
#Importing the libraries
from rake_nltk import Rake
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
#Loading the dataset
df=pd.read_csv('D:netflix_rs/netflix_titles.csv')
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


In [3]:
#Removing unnecessary columns
df=df.drop(columns=['show_id','country','date_added','release_year','rating','duration'])
df.head()

,type,title,director,cast,listed_in,description
0,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",Kids' TV,"With the help of three human allies, the Autob..."
3,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",Kids' TV,When a prison ship crash unleashes hundreds of...
4,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",Comedies,When nerdy high schooler Dani finally attracts...


In [4]:
#Barplot of tv or movies
df['type'].value_counts().plot('bar', figsize=[8,5], fontsize=15, color='navy') #There are a lot more movies than TV shows on Netflix


In [5]:
#Preprocessing the dataframe-(removing stopwords,punctation and lowercase all the letter )
stop = ['ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'him', 'each', 'the', 'themselves','an','of']
def preprocessdf(d):
    dataf=d.stack().str.replace(pat=r'([.;:!\'?,\"*()\[\]])',repl=' ')
    dataf=d.stack().str.replace(pat=r'(<br\s*/><br\s*/>)|(\-)|(\/)',repl=' ')
    dataf=d.stack().str.lower().unstack()
    dataf=d.stack().replace(to_replace=stop, value=" ",regex=True).unstack()
    return dataf

In [6]:
#Preprocessed dataframe
pdf=preprocessdf(df)
pdf

,type,title,director,cast,listed_in,description
0,Movie,Norm North: King Sized Adventure,"Richard Finn, Tim Maltby","Al Marriott, Andrew Toth, Bri Dobson, Cole H...","Children & Family Movies, Comedies",Before pl ning awesome wedding for his gr df...
1,Movie,J dino: Whatever it Takes,NaN,J dino Asporaat,St d-Up Comedy,J dino Asporaat riffs on challenges raisin...
2,TV Show,Tr sformers Prime,NaN,"Peter Cullen, Sumalee Mont o, Fr k Welker, Jef...",Kids' TV,"With help three hum allies, Autobots on..."
3,TV Show,Tr sformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Const ce Zimmer, K...",Kids' TV,When a prison ship crash unleashes hundreds ...
4,Movie,#realityhigh,Fern do Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",Comedies,When nerdy high schooler D i finally attracts ...
5,TV Show,Apaches,NaN,"Alberto Amm n, Eloy AzorÃ­n, VerÃ³nica Echegui...","Crime TV Shows, International TV Shows, Sp ish...",A young journalist is forced into a life cri...
6,Movie,Automata,Gabe IbÃ¡Ã±ez,"Antonio B deras, Dyl McDermott, Mel ie Griffi...","International Movies, Sci-Fi & F tasy, Thrillers","In a dystopi future, insur ce adjuster for ..."
7,Movie,Fabrizio Cop o: Solo pienso en mi,"Rodrigo Toro, Fr cisco Schultz",Fabrizio Cop o,St d-Up Comedy,Fabrizio Cop o takes audience participation to...
8,TV Show,Fire Chasers,NaN,NaN,"Docuseries, Science & Nature TV","As California's 2016 fire season rages, brave ..."
9,Movie,Good People,Henrik Ruben Genz,"James Fr co, Kate Hudson, Tom Wilkinson, Omar ...","Action & Adventure, Thrillers",A struggling couple c 't believe ir luck when...


In [7]:
#Raking the plot description for key phrases
import sys
reload(sys)
sys.setdefaultencoding('utf-8')
rdf=[]
for x in pdf['description']:
    r = Rake()

    # Extraction given the list of strings where each string is a sentence.
    r.extract_keywords_from_text(x)
    
    # To get keyword phrases ranked highest to lowest.
    r.get_ranked_phrases()
    rdf.append(r.get_ranked_phrases())
print rdf

In [8]:
#Adding the raked plot description to the dataframe
rake=(zip(rdf))
radf=pd.DataFrame(rake,columns=['Rdescription'])
radf['Rdescription']=radf['Rdescription'].str.get(0)
pdfs=pd.concat([pdf,radf],axis=1)
print display(pdfs)

,type,title,director,cast,listed_in,description,Rdescription
0,Movie,Norm North: King Sized Adventure,"Richard Finn, Tim Maltby","Al Marriott, Andrew Toth, Bri Dobson, Cole H...","Children & Family Movies, Comedies",Before pl ning awesome wedding for his gr df...,polar bear king must take back
1,Movie,J dino: Whatever it Takes,NaN,J dino Asporaat,St d-Up Comedy,J dino Asporaat riffs on challenges raisin...,j dino asporaat riffs
2,TV Show,Tr sformers Prime,NaN,"Peter Cullen, Sumalee Mont o, Fr k Welker, Jef...",Kids' TV,"With help three hum allies, Autobots on...",help three hum allies
3,TV Show,Tr sformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Const ce Zimmer, K...",Kids' TV,When a prison ship crash unleashes hundreds ...,prison ship crash unleashes hundreds decepticons
4,Movie,#realityhigh,Fern do Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",Comedies,When nerdy high schooler D i finally attracts ...,social media celebrity
5,TV Show,Apaches,NaN,"Alberto Amm n, Eloy AzorÃ­n, VerÃ³nica Echegui...","Crime TV Shows, International TV Shows, Sp ish...",A young journalist is forced into a life cri...,miguel s� � � � ez carral
6,Movie,Automata,Gabe IbÃ¡Ã±ez,"Antonio B deras, Dyl McDermott, Mel ie Griffi...","International Movies, Sci-Fi & F tasy, Thrillers","In a dystopi future, insur ce adjuster for ...",insur ce adjuster
7,Movie,Fabrizio Cop o: Solo pienso en mi,"Rodrigo Toro, Fr cisco Schultz",Fabrizio Cop o,St d-Up Comedy,Fabrizio Cop o takes audience participation to...,takes audience participation
8,TV Show,Fire Chasers,NaN,NaN,"Docuseries, Science & Nature TV","As California's 2016 fire season rages, brave ...",brave backcountry firefighters race
9,Movie,Good People,Henrik Ruben Genz,"James Fr co, Kate Hudson, Tom Wilkinson, Omar ...","Action & Adventure, Thrillers",A struggling couple c 't believe ir luck when...,struggling couple c


In [9]:
#Creating the bag of words
def bagofwords(tdf):
    bowl=[]
    for x in tdf:
        try:
            tds=(''.join(x.split()))
            tsc=tds.replace(',',' ')
            bowl.append(tsc)
        except:
            bowl.append('NaN')
        
    bowl=(bowl)
    return bowl

In [10]:
tr=bagofwords(pdfs['director'])
tt=bagofwords(pdfs['cast'])
ts=bagofwords(pdfs['listed_in'])
tu=(pdfs['Rdescription'])

In [11]:
#Getting bag of words and title of the tv show/movie together for the similiarity matrix.
gt=[]
for a,b,c,d in zip(tr,tt,ts,tu):
    gt.append(a + '' + b +'' + c + '' + d.decode('latin-1'))
gtr=pd.DataFrame(gt,columns=['bow'])    
#print display(gtr)
tbow=pd.concat([pdfs['title'],gtr],axis=1)
print display(tbow)

,title,bow
0,Norm North: King Sized Adventure,RichardFinn TimMaltbyAlMarriott AndrewToth Bri...
1,J dino: Whatever it Takes,NaNJdinoAsporaatStd-UpComedyj dino asporaat riffs
2,Tr sformers Prime,NaNPeterCullen SumaleeMonto FrkWelker JeffreyC...
3,Tr sformers: Robots in Disguise,NaNWillFriedle DarrenCriss ConstceZimmer Khary...
4,#realityhigh,FerndoLebrijaNestaCooper KateWalsh JohnMichael...
5,Apaches,NaNAlbertoAmmn EloyAzorÃ­n VerÃ³nicaEchegui Lu...
6,Automata,GabeIbÃ¡Ã±ezAntonioBderas DylMcDermott MelieGr...
7,Fabrizio Cop o: Solo pienso en mi,RodrigoToro FrciscoSchultzFabrizioCopoStd-UpCo...
8,Fire Chasers,NaNNaNDocuseries Science&NatureTVbrave backcou...
9,Good People,HenrikRubenGenzJamesFrco KateHudson TomWilkins...


In [12]:
#TFIDF vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(tbow['bow'])

#Getting the cosine similairity
cosinematrix=(cosine_similarity(tfidf_matrix, tfidf_matrix))
cosine1=pd.DataFrame(cosinematrix)
print display(cosine1)

,0,1,2,3,4,5,6,7,8,9,...,6224,6225,6226,6227,6228,6229,6230,6231,6232,6233
0,1.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
1,0.000000,1.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
2,0.000000,0.0,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
3,0.000000,0.0,0.000000,1.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
4,0.000000,0.0,0.000000,0.0,1.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
5,0.000000,0.0,0.000000,0.0,0.0,1.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.026876,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
6,0.000000,0.0,0.000000,0.0,0.0,0.000000,1.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.015718,0.0,0.0,0.000000,0.000000
7,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,1.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
8,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,1.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
9,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.010307,0.0,0.0,0.000000,0.000000


In [13]:
#Making a Recommender function with user inputted answers
titles=df['title']
#The recommender function to select top 10 most similiar movies/tv shows
def recommend():
    ri1=raw_input('Type a movie or tv show to get a recommendation ')
    ri2=raw_input('Select type to recommend (Movie,TV Show) ')
    
    if ri1  in (list(df['title'])) and ri2 =='TV Show' or ri2=='Movie':
        cos=cosine1
        num=df.loc[titles==ri1].index.values[0]#tvom
        top10=(titles[cos[num].nlargest(51).reset_index()['index']])
        top10=top10.iloc[1:51]
        filt=df.merge(top10,on='title')
        filt=filt[filt['type']==ri2]#ty
        recom=filt.iloc[1:11].reset_index()
        recom=recom.drop(['index','listed_in','director','cast'],axis=1)
        recoms=recom
        print display('Here are my recommendation for TV shows or movies for you to view based on similiarity')
        return display(recoms)     
    else:
        if ri1 not in (list(df['title'])) or ri2 !='TV Show' or ri2!= 'Movie':
            print display('It is not in the database or please check your spelling. Try again')
        ri1=raw_input('Type a movie or tv show to get a recommendation ')
        ri2=raw_input('Select type (Movie,TV Show) ')
        cos=cosine1
        num=df.loc[titles==ri1].index.values[0]
        top10=(titles[cos[num].nlargest(51).reset_index()['index']])
        top10=top10.iloc[1:51]
        filt=df.merge(top10,on='title')
        filt=filt[filt['type']==ri2]
        recom=filt.iloc[1:11].reset_index()
        recom=recom.drop(['index','listed_in','director','cast'],axis=1)
        recoms=recom
        print display('Here are my recommendation for TV shows or movies for you to view based on similiarity')
        return display(recoms)

In [14]:
recommend() 

Type a movie or tv show to get a recommendation Narcos
Select type to recommend (Movie,TV Show) TV Show


'Here are my recommendation for TV shows or movies for you to view based on similiarity'

,type,title,description
0,TV Show,Unbelievable,After a young woman is accused of lying about ...
1,TV Show,"Breakfast, Lunch & Dinner",Chef David Chang takes his insatiable curiosit...
2,TV Show,Narcos: Mexico,Witness the birth of the Mexican drug war in t...
3,TV Show,Miss Dynamite,"Wealthy, beautiful Valentina falls in love, on..."
4,TV Show,Dandy,A law professor is enlisted by Mexico's attorn...
5,TV Show,La Viuda Negra,"Beautiful and ruthless Griselda Blanco, known ..."
6,TV Show,DueÃ±os del paraÃ­so,When Anastasia Cardona's drug lord husband is ...
7,TV Show,Ugly Delicious,All the flavor. None of the BS. Star chef Davi...
8,TV Show,El Cartel,One man makes a fateful decision to get involv...
9,TV Show,El Cartel 2,Drug trafficker Pepe Cadena navigates the trea...


In [15]:
recommend()

Type a movie or tv show to get a recommendation Altered Carbon
Select type to recommend (Movie,TV Show) TV Show


'Here are my recommendation for TV shows or movies for you to view based on similiarity'

,type,title,description
0,TV Show,"Breakfast, Lunch & Dinner",Chef David Chang takes his insatiable curiosit...
1,TV Show,Nature's Great Events (2009),This series looks at how seasonal changes caus...
2,TV Show,What If?,Four individuals at a crossroads in life are g...
3,TV Show,All Hail King Julien: Exiled,"Julien's been dethroned, but loyal friends and..."
4,TV Show,Somewhere Between,A local news producer is given one chance to r...
5,TV Show,Ugly Delicious,All the flavor. None of the BS. Star chef Davi...
6,TV Show,Fast & Furious Spy Racers,A government agency recruits teen driver Tony ...
7,TV Show,Our Planet,Experience our planet's natural beauty and exa...
8,TV Show,Marvel's Iron Fist,Danny Rand resurfaces 15 years after being pre...
9,TV Show,Person of Interest,Ex-CIA agent John Reese and enigmatic billiona...


In [16]:
recommend()

Type a movie or tv show to get a recommendation Black Panther
Select type to recommend (Movie,TV Show) Movie


'Here are my recommendation for TV shows or movies for you to view based on similiarity'

,type,title,description
0,Movie,13TH,"In this thought-provoking documentary, scholar..."
1,Movie,Derren Brown: Sacrifice,Illusionist Derren Brown concocts a psychologi...
2,Movie,Men in Black II,Will Smith and Tommy Lee Jones reprise their r...
3,Movie,Supergirl,"When she loses her city's vital power supply, ..."
4,Movie,Muran,When jingle composer Nanda's car breaks down o...
5,Movie,The Score,"Ready-to-retire safecracker Nick, flamboyant f..."
6,Movie,Be Somebody,"To escape the stress of fame, teen pop star Jo..."
7,Movie,The Adventures of Tintin,This 3-D motion capture adapts Georges Remi's ...
8,Movie,Enter the Dragon,"Recruited by a British agent, Shaolin master L..."
9,Movie,Mutiny on the Bounty,Marlon Brando gives a nuanced performance as t...


In [17]:
recommend()

Type a movie or tv show to get a recommendation Men in Black
Select type to recommend (Movie,TV Show) Movie


'Here are my recommendation for TV shows or movies for you to view based on similiarity'

,type,title,description
0,Movie,Bitcoin Heist,"A unconventional, efficient Interpol special a..."
1,Movie,Private Life,"Struggling to conceive, a couple in their 40s ..."
2,Movie,Incoming,When an imprisoned terrorist cell hijacks a hi...
3,Movie,Scorpion King 5: Book of Souls,The Scorpion King joins forces with a Nubian p...
4,Movie,Men in Black II,Will Smith and Tommy Lee Jones reprise their r...
5,Movie,Supergirl,"When she loses her city's vital power supply, ..."
6,Movie,Superman Returns,"When Superman returns to Metropolis, he finds ..."
7,Movie,The Bounty Hunter,Bounty hunter Milo Boyd finds his latest assig...
8,Movie,Brooklyn's Finest,Three cops find themselves hurtling toward the...
9,Movie,Phantom Boy,A boy whose illness allows him to float outsid...
